# Similar objects in simbad, GEV and TEV catalogs
First, we import all the packages that we will need.

In [1]:
import warnings
import pandas as pd
import numpy as np
from getsimbad import get_simbad_data
from gevtev import compare_gev_tev_data
from definitions import *

warnings.filterwarnings("ignore")

This constant determines a threshold for the similarity between the objects:
$$|glat_{1} - glat_{2}| <= \epsilon$$
$$\epsilon = 1,2*10^{-1}$$

In [2]:
_epsilon = 0.12

The cell above loads data from SIMBAD catalog obtained by requests 'Be', 'O', 'B' and 'plsr'. If the file "data/symbad.txt" does not exist, run getsimbad.py.

In [3]:
try:
    pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', encoding='utf-8')
    del pd_simbad['Unnamed: 0']
except FileNotFoundError:
    pd_simbad = get_simbad_data()


In [4]:
pd_simbad.head()

,s_MAIN_ID,s_RA,s_DEC,s_RA_PREC,s_DEC_PREC,s_COO_ERR_MAJA,s_COO_ERR_MINA,s_COO_ERR_ANGLE,s_COO_QUAL,s_COO_WAVELENGTH,...,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_class,s_glat,s_glon,s_nan_spectra,s_zero_spectra
0,b'HD 201433A',21 08 38.894,+30 12 20.27,7.0,7.0,26.0000,26.0000,90.0,B,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-11.774311,76.107743,False,False
1,b'* nu. For',02 04 29.4462,-29 17 48.491,14.0,14.0,0.1834,0.1942,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-73.777993,224.887601,False,False
2,b'* phi Her',16 08 46.1668,+44 56 05.732,14.0,14.0,0.3710,0.3754,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,47.107400,70.847825,False,False
3,b'V* CG And',00 00 43.6345,+45 15 12.002,14.0,14.0,0.0550,0.0422,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-16.702455,113.649358,False,False
4,b'HD 133029',15 00 38.7184,+47 16 38.793,14.0,14.0,0.0472,0.0421,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,57.710074,80.191942,False,False


In [5]:
s_spectrum_columns = list_s_spectrum_columns()

In [6]:
pd_simbad['s_nan_spectra'] = pd_simbad[s_spectrum_columns].isna().all(axis=1)

In [7]:
pd_simbad['s_zero_spectra'] = ~(pd_simbad[s_spectrum_columns] != 0).any(axis = 1)

We load data common for GEV and TEV catalogs.

In [8]:
common_data, only_tev_data, only_gev_data = compare_gev_tev_data(0.01)

tev_source_id
tev_common_name
tev_gamma_names
tev_fermi_names
tev_other_names
tev_where
tev_classes
tev_discoverer
tev_seen_by
tev_discovery_date
tev_tevcat_id
tev_tevcat2_id
tev_tevcat_name
tev_tgevcat_id
tev_tgevcat_name
tev_reference_id
tev_significance
tev_livetime
tev_ra
tev_dec
tev_glon
tev_glat
tev_pos_ra
tev_pos_dec
tev_pos_glon
tev_pos_glat
tev_pos_err
tev_morph_type
tev_morph_sigma
tev_morph_sigma_err
tev_morph_sigma2
tev_morph_sigma2_err
tev_morph_pa
tev_morph_pa_err
tev_morph_pa_frame
tev_spec_type
tev_spec_pl_norm
tev_spec_pl_norm_err
tev_spec_pl_norm_err_sys
tev_spec_pl_index
tev_spec_pl_index_err
tev_spec_pl_index_err_sys
tev_spec_pl_e_ref
tev_spec_pl2_flux
tev_spec_pl2_flux_err
tev_spec_pl2_flux_err_sys
tev_spec_pl2_index
tev_spec_pl2_index_err
tev_spec_pl2_index_err_sys
tev_spec_pl2_e_min
tev_spec_pl2_e_max
tev_spec_ecpl_norm
tev_spec_ecpl_norm_err
tev_spec_ecpl_norm_err_sys
tev_spec_ecpl_index
tev_spec_ecpl_index_err
tev_spec_ecpl_index_err_sys
tev_spec_ecpl_e_cut
tev

The following cells compare distances from a common catalog of GEV and TEV and SIMBAD catalog by translating SIMBAD coordinates to glan and glot due to astropy module. If their relative distance is smaller than $\epsilon$ they are associated to each another.

In [9]:
def create_vectors_common_gevtev_simbad_old(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: two vectors where each index of TEV(GEV) is associated with its indev in GEV(TEV).
    If such objects does not existe, the corresponding vector coordinate is equal to -1
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    # в данной реализации:
    # если для объекта i gevtev по расстоянию подходит несколько объектов из simbad, 
    # то привязан он будет к последнему
    for i in range(len(glat_gevtev)):
        for j in range(len(glat_simbad)):
            if ((np.abs(glat_gevtev[i] - glat_simbad[j]) < epsilon) 
                and (np.abs(glon_gevtev[i] - glon_simbad[j]) < epsilon)) :
                    C_associations_gevtev[i] = j
                    C_associations_simbad[j] = i
    return C_associations_gevtev, C_associations_simbad

In [10]:
def create_matrix_common_gevtev_simbad(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix with association: m[i][j] = True if object i from gevtev
            is close enough to object j from simbad
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    glat_dif_matrix = np.dot(np.vstack((glat_gevtev, -np.ones_like(glat_gevtev))).T,
                            np.vstack((np.ones_like(glat_simbad), glat_simbad)))
    glon_dif_matrix = np.dot(np.vstack((glon_gevtev, -np.ones_like(glon_gevtev))).T,
                            np.vstack((np.ones_like(glon_simbad), glon_simbad)))
    pairs_matrix = np.logical_and(np.abs(glat_dif_matrix) < epsilon, 
                                np.abs(glon_dif_matrix) < epsilon)
    return pairs_matrix

In [11]:
def create_simbad_pandas_for_gevtev(gevtev, simbad, epsilon, catalog="tev"):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a pandas with associating IDs and classes from simbad in the gevtev order
    
    gevtev - a pandas dataFrame of common GeV and TeV 
    simbad - a pandas dataFrame of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    if (catalog.find("tev") != -1):
        glat_gevtev = gevtev[catalog+'_glat']
        glon_gevtev = gevtev[catalog+'_glon']
    else:
        glat_gevtev = gevtev[catalog+'_GLAT']
        glon_gevtev = gevtev[catalog+'_GLON']        

    glat_simbad = simbad['s_glat']
    glon_simbad = simbad['s_glon']
    
    C_associations_gevtev = -1.0 * np.ones((len(glat_gevtev)))
    C_associations_simbad = -1.0 * np.ones((len(glat_simbad)))
    
    glat_dif_matrix = np.dot(np.vstack((glat_gevtev, -np.ones_like(glat_gevtev))).T,
                            np.vstack((np.ones_like(glat_simbad), glat_simbad)))
    glon_dif_matrix = np.dot(np.vstack((glon_gevtev, -np.ones_like(glon_gevtev))).T,
                            np.vstack((np.ones_like(glon_simbad), glon_simbad)))
    pairs_matrix = np.logical_and(np.abs(glat_dif_matrix) < epsilon, 
                                np.abs(glon_dif_matrix) < epsilon)

    
    s_classes_list_of_lists = []
    s_main_id_list_of_lists = []
    s_simbad = [] # 0 - not in simbad, 2 - in simbad without spectra, 1 - in simbad with spectra
    for i in pairs_matrix:
        association_simbad_numbers = np.where(i)
        s_classes_pandas = simbad.iloc[association_simbad_numbers]['s_class']
        s_main_id_pandas = simbad.iloc[association_simbad_numbers]['s_MAIN_ID']
        if len(s_classes_pandas.unique()) == 1:
            s_classes_list = s_classes_pandas.unique()
            s_main_id_list = s_main_id_pandas.values
        else:
            s_classes_list = s_classes_pandas.values
            s_main_id_list = s_main_id_pandas.values
        s_classes_list_of_lists.append(s_classes_list)
        s_main_id_list_of_lists.append(s_main_id_list)
        if len(s_classes_list) == 0:
            s_simbad.append(0)
        else:
            if simbad.iloc[association_simbad_numbers]['s_nan_spectra'].any():
                s_simbad.append(2)
            else:
                s_simbad.append(1)
        
    simbad_no_spectra_columns = ['s_MAIN_ID', 's_class', 's_simbad'] 
    simbad_no_spectra_pandas = pd.DataFrame(columns = simbad_no_spectra_columns)   
    simbad_no_spectra_pandas['s_MAIN_ID'] = s_main_id_list_of_lists
    simbad_no_spectra_pandas['s_class'] = s_classes_list_of_lists
    simbad_no_spectra_pandas['s_simbad'] = s_simbad
    return simbad_no_spectra_pandas

In [12]:
def create_common_data(data_gevtev, data_simbad, pairs_matrix, catalog="tev"):
    """The fonction adds objects found both in GeV, TeV and simbad.
    """
    vector_association = np.where(np.sum(pairs_matrix, axis=0) > 0)[0]
    pd_common_gevtev = pd.DataFrame()
    for i in vector_association:
        data_gevtev_join = (pairs_matrix[:, i] > 0)*(i + 1) - 1
        data_gevtev["join"] = data_gevtev_join
        pd_common_gevtev0 = pd.merge(data_gevtev, data_simbad, right_index=True, left_on='join', how='inner')
        if (len(pd_common_gevtev)):
            pd_common_gevtev = pd_common_gevtev.append(pd_common_gevtev0)
            del pd_common_gevtev["join"]
            del data_gevtev["join"]
        else:
            pd_common_gevtev = pd_common_gevtev0.copy()
    
    
    if (catalog.find("tev") != -1):    
        array_non_duplicate = [catalog+'_glat', catalog+'_glon', 's_glat','s_glon']
    else:
        array_non_duplicate = [catalog+'_GLAT', catalog+'_GLON', 's_glat','s_glon']
    pd_common_gevtev = pd_common_gevtev.drop_duplicates(array_non_duplicate)
    pd_common_gevtev = pd_common_gevtev.reset_index()

    #df_common = pd.DataFrame(data = data, columns = namefinal)
    return pd_common_gevtev

In [13]:
matrix = create_matrix_common_gevtev_simbad(common_data, pd_simbad, _epsilon)
pd_common_gevtevsimbad = create_common_data(common_data, pd_simbad, matrix)
print("There are " + str(len(pd_common_gevtevsimbad)) + " common objects in the three catalogs.")

There are 72 common objects in the three catalogs.


In [14]:
pd_simbad_for_gevtev = create_simbad_pandas_for_gevtev(common_data, pd_simbad, _epsilon)
pd_common_gevtev = pd.merge(common_data, pd_simbad_for_gevtev, how='inner', left_index=True, right_index=True)

All gevtev objects with simbad columns containing in s_simbad :
- 0 - if near this gevtev object any simbad object is found
- 1 - if near this gevtev object a simbad object is found and its spectrum is given
- 2 - if near this gevtev object a simbad object is found and its spectrum is not given

in s_MAIN_ID:

- list of simbad names in corresponding objects

in s_class:

If all corresponding objects have the same class - its name
if not - classes in the same order as names in s_MAIN_ID

In [15]:
pd_common_gevtev

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
0,19,b'0FGL J0007.4+7303',b'1FGL J0007.0+7303 ',b'1FHL J0007.3+7303 ',b'2FGL J0007.0+7303 ',b'LAT PSR J0007+7303 ',b' ',b'1AGL J0006+7311',b' ',b'EGR J0008+7308 ',...,b'pl',b'rC5JCj',227,b'TeV J0006+729',1,b'TeV J0006+7259',b'gal',[],[],0
1,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'pl',b'70hRXZ',213,b'TeV J0013-188',2,b'TeV J0013-1853',b'egal',[],[],0
2,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'pl',b'xCyAjF',209,b'TeV J0025+641',3,b'TeV J0025+6410',b'gal',[],[],0
3,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'none',b'PKJZqs',239,b'TeV J0033-193',4,b'TeV J0033-1921',b'egal',[],[],0
4,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'none',b'eNYfrm',229,b'TeV J0035+598',5,b'TeV J0035+5947',b'egal',[],[],0
5,99,b' ',b'1FGL J0047.3-2512 ',b' ',b'2FGL J0047.0-2516 ',b'NGC 253 ',b' ',b' ',b' ',b' ',...,b'none',b'sK2rQU',195,b'TeV J0047-253',6,b'TeV J0047-2517',b'egal',[],[],0
6,193,b'0FGL J0136.6+3903',b'1FGL J0136.5+3905 ',b'1FHL J0136.5+3906 ',b'2FGL J0136.5+3905 ',b'B3 0133+388 ',b' ',b' ',b' ',b' ',...,b'none',b'7WhQrw',244,b'TeV J0136+391',7,b'TeV J0136+3905',b'egal',[],[],0
7,214,b' ',b' ',b'1FHL J0152.6+0148 ',b'2FGL J0152.6+0148 ',b'PMN J0152+0146 ',b' ',b' ',b' ',b' ',...,b'pl',b'tKGVcC',156,b'TeV J0152+017',8,b'TeV J0152+0146',b'egal',[],[],0
8,241,b' ',b'1FGL J0205.6+6449 ',b'1FHL J0205.7+6448 ',b'2FGL J0205.8+6448 ',b'PSR J0205+6449 ',b' ',b' ',b' ',b' ',...,b'pl',b'2I2TuY',252,b'TeV J0209+648',9,b'TeV J0205+6451',b'gal',[],[],0
9,273,b'0FGL J0220.9+3607',b'1FGL J0221.0+3555 ',b'1FHL J0221.1+3555 ',b'2FGL J0221.0+3555 ',b'B0218+357 ',b' ',b' ',b' ',b' ',...,b'none',b'hJnA37',254,b'TeV J0218+359',10,b'TeV J0221+3556',b'egal',[],[],0


ALL gevtev objects found in simbad

In [16]:
pd_common_gevtev[pd_common_gevtev['s_simbad']>0]

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
13,319,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,b'pl',b'fW749W',89,b'TeV J0240+612',14,b'TeV J0240+6115',b'gal',"[b'LS I +61 303', b'LS I +61 303']",[B0Ve],1
29,696,b'0FGL J0617.4+2234',b'1FGL J0617.2+2233 ',b'1FHL J0617.2+2234e',b'2FGL J0617.2+2234e',b'IC 443 ',b' ',b'1AGL J0617+2236',b'3EG J0617+2238',b'EGR J0617+2238 ',...,b'pl',b'9xMcgD',120,b'TeV J0616+225',29,b'TeV J0616+2230',b'gal',[b'HD 43582'],[B0Ve],1
30,701,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'pl',b'9xMcgD',120,b'TeV J0616+225',29,b'TeV J0616+2230',b'gal',[b'HD 43582'],[B0Ve],1
42,1119,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',...,b'none',b'wN7SNs',194,b'TeV J0955+696',40,b'TeV J0955+6940',b'egal',"[b'[CHP2004] J095549.8+693934', b'[MTK2001b] 8...",[B0Ve],2
49,1162,b' ',b' ',b' ',b'2FGL J1016.5-5858 ',b'PSR J1016-5857 ',b' ',b' ',b'3EG J1013-5915',b' ',...,b'pl',b'5FlzLA',237,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',[b'2FGL J1019.0-5856'],[B0Ve],2
50,1172,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,b'pl',b'5FlzLA',237,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',[b'2FGL J1019.0-5856'],[B0Ve],2
91,1962,b' ',b'1FGL J1614.7-5138c',b'1FHL J1615.3-5146e',b'2FGL J1615.2-5138 ',b' ',b' ',b' ',b' ',b' ',...,b'pl',b'ChXqoJ',234,b'TeV J1614-518',83,b'TeV J1614-5149',b'gal',"[b'MSX6C G331.5487-00.5364', b'MSX6C G331.5487...","[Be, B]",1
95,2017,b' ',b' ',b' ',b'2FGL J1632.4-4820c',b'PSR J1632-4818 ',b' ',b' ',b' ',b' ',...,b'pl2',b'p9DNQ0',105,b'TeV J1632-478',86,b'TeV J1632-4749',b'gal',[b'2FHL J1633.5-4746e'],[B0Ve],2
96,2019,b' ',b'1FGL J1632.7-4733c',b'1FHL J1633.0-4746e',b'2FGL J1632.4-4753c',b'HESS J1632-478 ',b' ',b' ',b' ',b' ',...,b'pl2',b'p9DNQ0',105,b'TeV J1632-478',86,b'TeV J1632-4749',b'gal',[b'2FHL J1633.5-4746e'],[B0Ve],2
117,2222,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',"[b'SSTGC 519103', b'SSTGC 519103', b'GCIRS 1W...","[Be, B, B, O]",1


In [17]:
pd_common_gevtevsimbad.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_pl_norm_err,tev_spec_pl_norm_err_sys,tev_spec_theta,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where
0,91,b' ',b'1FGL J1614.7-5138c',b'1FHL J1615.3-5146e',b'2FGL J1615.2-5138 ',b' ',b' ',b' ',b' ',b' ',...,3.400000e-13,1.760000e-12,0.49,b'pl',b'ChXqoJ',234,b'TeV J1614-518',83,b'TeV J1614-5149',b'gal'
1,117,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
2,178,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,1.200000e-13,1.500000e-13,0.15,b'pl',b'F0Luol',193,b'TeV J1930+188',136,b'TeV J1930+1852',b'gal'
3,160,b' ',b' ',b' ',b' ',b'LAT PSR J1838-0537 ',b' ',b' ',b' ',b' ',...,1.300000e-12,2.000000e-13,0.70,b'pl',b'amNIwH',142,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal'
4,117,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'


The next cell chooses several columns from a dataFrame to make sure that the output is correct(objects have the same coordinates etc.)

In [18]:
columns_to_compare = ['s_glat',
                     's_glon',
                     'gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     's_class',
                     'gev_CLASS1',
                     'tev_classes']
pd_common_gevtevsimbad[columns_to_compare].head()

,s_glat,s_glon,gev_ASSOC_TEV,tev_glat,gev_GLAT,tev_glon,gev_GLON,s_class,gev_CLASS1,tev_classes
0,-0.536628,331.548422,b'HESS J1614-518 ',-0.573211,-0.659245,331.512299,331.659454,Be,b'spp ',b'unid'
1,-0.011453,359.985525,b' ',-0.046162,-0.020071,359.944244,359.863953,Be,b' ',b'unid'
2,0.280229,54.095263,b' ',0.260453,0.577399,54.094742,53.656170,B,b' ',"b'pwn,snr'"
3,-0.198384,26.857782,b' ',-0.197604,0.206789,26.795113,26.501699,B,b'PSR ',b'unid'
4,-0.047327,359.945163,b' ',-0.046162,-0.020071,359.944244,359.863953,B,b' ',b'unid'


In [19]:
columns_to_compare_gevtev = ['gev_ASSOC_TEV',
                     'tev_glat', 
                     'gev_GLAT',
                     'tev_glon',
                     'gev_GLON',
                     'gev_CLASS1',
                     'tev_classes']

In [20]:
common_data.to_csv("gevtev.txt", sep='\t')

The next cells find common values in simbad-tev

In [21]:
matrix = create_matrix_common_gevtev_simbad(only_tev_data, pd_simbad, _epsilon)
pd_common_onlytevsimbad = create_common_data(only_tev_data, pd_simbad, matrix)
print("There are " + str(len(pd_common_onlytevsimbad)) + " common objects in tev-simbad catalogs.")

There are 123 common objects in tev-simbad catalogs.


The next cells find common values in simbad-gev

In [22]:
matrix = create_matrix_common_gevtev_simbad(only_gev_data, pd_simbad, _epsilon, catalog="gev")
pd_common_onlygevsimbad = create_common_data(only_gev_data, pd_simbad, matrix, catalog="gev")
print("There are " + str(len(pd_common_onlygevsimbad)) + " common objects in gev-simbad catalogs.")

There are 155 common objects in gev-simbad catalogs.


In [23]:
pd_common_gevtevsimbad.to_csv("data/gevtevsimbad.txt", sep='\t')
pd_common_onlygevsimbad.to_csv("data/gevsimbad.txt", sep='\t')
pd_common_onlytevsimbad.to_csv("data/tevsimbad.txt", sep='\t')
common_data.to_csv("data/gevtev.txt", sep='\t')

In [24]:
pd_common_gevtev.to_csv("data/gevtev_simbadclasses.txt", sep='\t')

In [25]:
print(str(np.sum(pd_simbad['s_nan_spectra'])) + ' simbad objects do not have simbad spectra. In total '+ str(len(pd_simbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_simbad['s_zero_spectra'])) + ' simbad objects have zero simbad spectra. In total '+ str(len(pd_simbad['s_zero_spectra'] )) + ' objects.')
print(str(np.sum(pd_common_gevtevsimbad['s_nan_spectra'])) + ' gevtevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_gevtevsimbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_common_onlygevsimbad['s_nan_spectra'])) + ' gevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_onlygevsimbad['s_nan_spectra'])) + ' objects.')
print(str(np.sum(pd_common_onlytevsimbad['s_nan_spectra'])) + ' tevsimbad objects do not have simbad spectra. In total '+ str(len(pd_common_onlytevsimbad['s_nan_spectra'])) + ' objects.')

1308 simbad objects do not have simbad spectra. In total 11625 objects.
0 simbad objects have zero simbad spectra. In total 11625 objects.
55 gevtevsimbad objects do not have simbad spectra. In total 72 objects.
63 gevsimbad objects do not have simbad spectra. In total 155 objects.
56 tevsimbad objects do not have simbad spectra. In total 123 objects.


In [26]:
no_spectra_gevtevsimbad_objects = pd_common_gevtevsimbad[pd_common_gevtevsimbad['s_nan_spectra']]['s_MAIN_ID'].unique()

In [27]:
no_spectra_gevtevsimbad_objects.sort()
for i in no_spectra_gevtevsimbad_objects:
    print(i)

b'2FGL J1019.0-5856'
b'2FHL J1633.5-4746e'
b'CXOM82 J095550.5+694022'
b'CXOU J095510.7+693955'
b'CXOU J095527.1+693923'
b'CXOU J095527.2+694050'
b'CXOU J095531.8+693957'
b'CXOU J095534.5+693823'
b'CXOU J095544.8+694100'
b'CXOU J095546.1+694026'
b'CXOU J095546.5+694040'
b'CXOU J095547.4+694059'
b'CXOU J095547.5+694120'
b'CXOU J095548.9+694010'
b'CXOU J095549.1+694036'
b'CXOU J095550.1+694034'
b'CXOU J095550.2+694039'
b'CXOU J095550.5+694056'
b'CXOU J095550.6+694043'
b'CXOU J095550.7+694039'
b'CXOU J095550.9+694044'
b'CXOU J095551.0+694050'
b'CXOU J095551.2+694012'
b'CXOU J095551.3+694043'
b'CXOU J095551.4+694035'
b'CXOU J095552.2+694042'
b'CXOU J095552.4+694024'
b'CXOU J095552.7+694045'
b'CXOU J095552.8+694020'
b'CXOU J095553.0+694048'
b'CXOU J095553.2+694052'
b'CXOU J095553.3+694101'
b'CXOU J095553.9+694045'
b'CXOU J095554.1+694039'
b'CXOU J095554.9+694038'
b'CXOU J095555.1+694027'
b'CXOU J095600.7+694107'
b'CXOU J095601.7+694308'
b'CXOU J095608.0+694139'
b'CXOU J095614.7+694249'
b'CXO